In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load Data and Import Libraries

In [3]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


In [5]:
!unzip /content/drive/My\ Drive/Questions.zip


Archive:  /content/drive/My Drive/Questions.zip
  inflating: Questions.csv           


In [6]:
!unzip /content/drive/My\ Drive/Tags.zip

Archive:  /content/drive/My Drive/Tags.zip
  inflating: Tags.csv                


In [8]:
question_df = pd.read_csv('Questions.csv', encoding='latin-1')
tags_df = pd.read_csv('Tags.csv')

In [17]:
pd.set_option('display.max_colwidth',200)

In [13]:

question_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Machine Learning, fight!""</a> that discussed some of the differences between the two fields. <a href=""http://andrewgelman.com/2008/12/machine_learnin/"">Andrew Gelman responded favorably to this</a>:</p>\n\n<p>Simon Blomberg: </p>\n\n<blockquote>\n <p>From R's fortunes\n package: To paraphrase provoca..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,"<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\nareas are a lot larger than condensed\nurban areas. Is there a need to account for the area size difference?</li>\n<li>if let's say I have census data\ndating back to 4 - 5 census periods,\nhow far can i forecast it into the\nfuture?</li>\n<li>if some of the census zone change\nlightly in boundaries..."
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests. It seems that students easily learn how to perform the calculations required by a given test but get hung up on interpreting the results. Many computerized tools report test results in terms of ""p values"" or ""t values"".</p>\n\n<p>How would you explain the following points to college students taki..."
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth rate in Denmark;</li>\n<li>number of priests in America and alcoholism;</li>\n<li>in the start of the 20th century it was noted that there was a strong correlation between 'Number of radios' and 'Number of people in Insane Asylums'</li>\n<li>and my favorite: <a href=""http://en.wikipedia.org/wiki/Fil..."


# Text Cleaning

In [14]:
def cleaner(text):
  # take off html tags
  text = BeautifulSoup(text).get_text()

  # fetch alphanumeric characters
  text = re.sub("[^a-zA-Z]", " ",text)

  #convert text to lower case
  text = text.lower()

  #split text into tokens to remove whitespaces
  tokens = text.split()

  return " ".join(tokens)

In [15]:
question_df['cleaned_text'] = question_df['Body'].apply(cleaner)

In [18]:
question_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,cleaned_text
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...


# Merge Tags with Questions

In [19]:
tags_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [21]:
#count of unique tags
len(tags_df['Tag'].unique())

1315

In [22]:
tags_df['Tag'].value_counts()

r                       13236
regression              10959
machine-learning         6089
time-series              5559
probability              4217
                        ...  
bayesian-anova              1
propensity                  1
netflix-prize               1
hierarchical-softmax        1
gmm                         1
Name: Tag, Length: 1315, dtype: int64

In [23]:
# remove "-" from the tags
tags_df.Tag = tags_df['Tag'].apply(lambda x:re.sub("-"," ",x))

In [28]:
tags_df['Tag'].values

array(['bayesian', 'prior', 'elicitation', ..., 'normal distribution',
       'estimation', 'median'], dtype=object)

In [32]:
# group tags by Id wise
tags_df = tags_df.groupby('Id').apply(lambda x:x['Tag'].values).reset_index(name='tags')

In [33]:
tags_df.head()

,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open source]"
3,4,"[distributions, statistical significance]"
4,6,[machine learning]


In [34]:
df = pd.merge(question_df,tags_df,how='inner',on='Id')

In [35]:
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,cleaned_text,tags
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...,[machine learning]
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...,"[forecasting, population, census]"
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning,"[bayesian, frequentist]"
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...,"[correlation, teaching]"


In [36]:
df = df[['Id','Body','cleaned_text','tags']]
df.head()

,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...,[machine learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...,"[correlation, teaching]"


In [37]:
df.shape

(85085, 4)

# Dataset Preparation

In [51]:
freq = {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] = freq[j]+1
    else:
      freq[j] = 1


In [52]:
# sort the dictionary in descending order
freq = dict(sorted(freq.items(), key=lambda x:x[1],reverse=True))  

In [53]:
freq.items()

dict_items([('r', 13236), ('regression', 10959), ('machine learning', 6089), ('time series', 5559), ('probability', 4217), ('hypothesis testing', 3869), ('self study', 3732), ('distributions', 3501), ('logistic', 3316), ('classification', 2881), ('correlation', 2871), ('statistical significance', 2666), ('bayesian', 2656), ('anova', 2505), ('normal distribution', 2181), ('multiple regression', 2054), ('mixed model', 1998), ('clustering', 1952), ('neural networks', 1897), ('mathematical statistics', 1888), ('confidence interval', 1776), ('categorical data', 1703), ('generalized linear model', 1614), ('variance', 1576), ('data visualization', 1549), ('estimation', 1533), ('forecasting', 1422), ('t test', 1418), ('pca', 1395), ('sampling', 1363), ('cross validation', 1344), ('repeated measures', 1335), ('spss', 1296), ('svm', 1283), ('chi squared', 1261), ('maximum likelihood', 1209), ('predictive models', 1189), ('multivariate analysis', 1116), ('survival', 1081), ('references', 1076), (

In [55]:
# Top 10 most frequent tags
common_tags = list(freq.keys())[:10]
common_tags

['r',
 'regression',
 'machine learning',
 'time series',
 'probability',
 'hypothesis testing',
 'self study',
 'distributions',
 'logistic',
 'classification']

In [58]:
x=[]
y=[]

for i in range(len(df['tags'])):

  temp=[]
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)
  if(len(temp)>1):
    x.append(df['cleaned_text'][i])
    y.append(temp)

In [59]:
len(x)

11106

In [61]:
y[:10]

[['r', 'time series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis testing'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'time series'],
 ['r', 'time series', 'self study'],
 ['probability', 'hypothesis testing'],
 ['r', 'regression'],
 ['r', 'regression'],
 ['regression', 'logistic']]

In [62]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

y = mlb.fit_transform(y)
y.shape

(11106, 10)

In [63]:
y[0,:]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [64]:
mlb.classes_

array(['classification', 'distributions', 'hypothesis testing',
       'logistic', 'machine learning', 'probability', 'r', 'regression',
       'self study', 'time series'], dtype=object)

In [65]:
# splitting the data
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.2,random_state=0,shuffle=True)

# Text Representation

In [66]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(x_train)

In [67]:
x_tokenizer.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'a': 4,
 'of': 5,
 'is': 6,
 'and': 7,
 'in': 8,
 'l': 9,
 'x': 10,
 'for': 11,
 'that': 12,
 'data': 13,
 'this': 14,
 't': 15,
 'have': 16,
 'y': 17,
 'with': 18,
 'model': 19,
 'it': 20,
 'are': 21,
 'be': 22,
 'my': 23,
 'as': 24,
 'on': 25,
 'e': 26,
 'p': 27,
 'if': 28,
 'can': 29,
 'n': 30,
 'but': 31,
 'not': 32,
 'm': 33,
 'or': 34,
 'r': 35,
 'how': 36,
 'regression': 37,
 'c': 38,
 'am': 39,
 's': 40,
 'from': 41,
 'test': 42,
 'what': 43,
 'would': 44,
 'b': 45,
 'so': 46,
 'time': 47,
 'there': 48,
 'using': 49,
 'which': 50,
 'an': 51,
 'do': 52,
 'one': 53,
 'each': 54,
 'value': 55,
 'use': 56,
 'by': 57,
 'some': 58,
 'variables': 59,
 'like': 60,
 'variable': 61,
 'we': 62,
 'at': 63,
 'na': 64,
 'any': 65,
 'f': 66,
 'distribution': 67,
 'two': 68,
 'values': 69,
 'set': 70,
 'you': 71,
 'all': 72,
 'function': 73,
 'fit': 74,
 'd': 75,
 'beta': 76,
 'question': 77,
 'then': 78,
 'mean': 79,
 'me': 80,
 'know': 81,
 'where': 82,
 'when'

In [68]:
len(x_tokenizer.word_index)

25312

In [70]:
thresh = 3

count = 0

for key,value in x_tokenizer.word_counts.items():
  if value>=thresh:
    count = count+1

print(count)


12574


In [71]:
x_tokenizer.word_counts.items()

odict_items([('i', 53875), ('am', 5963), ('testing', 603), ('for', 17220), ('cointegration', 77), ('using', 4846), ('the', 73528), ('johansen', 22), ('test', 5402), ('have', 10623), ('seen', 259), ('questions', 859), ('like', 3751), ('how', 6118), ('to', 35727), ('interpret', 361), ('results', 1645), ('but', 6858), ('when', 2631), ('interpreting', 85), ('mine', 43), ('some', 3885), ('doubts', 37), ('in', 20636), ('my', 8504), ('r', 6220), ('since', 996), ('so', 4973), ('cannot', 438), ('form', 537), ('a', 35715), ('stationary', 277), ('series', 2567), ('it', 9629), ('is', 28025), ('same', 2034), ('and', 26017), ('there', 4937), ('combination', 209), ('implies', 65), ('that', 15891), ('are', 9001), ('zero', 648), ('cointegrating', 14), ('vectors', 251), ('does', 1925), ('mean', 2741), ('data', 13524), ('not', 6785), ('cointegrated', 28), ('therefore', 337), ('build', 399), ('vecm', 32), ('please', 791), ('find', 1782), ('below', 947), ('ca', 58), ('jo', 19), ('canada', 3), ('type', 1199

In [73]:
# prepare tokenizer again where tokens appear more than 3 times are considered

x_tokenizer = Tokenizer(num_words=count, oov_token='unk')

#prepare vocabulary
x_tokenizer.fit_on_texts(x_train)

Now that we have encoded every token to an integer, let's convert the text sequences to integer sequences. After that we will pad the integer sequences to the maximum sequence length, i.e., 100.

In [74]:
# maximum sequence length allowed
max_len = 100

# convert text sequence into integer sequences
x_train_seq = x_tokenizer.texts_to_sequences(x_train)
x_valid_seq = x_tokenizer.texts_to_sequences(x_valid)

#padding up with zero

x_train_seq = pad_sequences(x_train_seq, padding='post', maxlen=max_len)
x_valid_seq = pad_sequences(x_valid_seq, padding='post', maxlen=max_len)

In [75]:
# Since we are padding the sequences with zeros, we must increment the vocabulary size by one.
x_voc_size = x_tokenizer.num_words + 1
x_voc_size

12575

In [76]:
x_train_seq[0]

array([1953, 5711,  416, 2023,    1,  226, 1747, 3740,  609,   43,  181,
       1953,  372,   19,  100,  416,    9, 1747, 3839,  238,   27,   27,
         27,   27,   27,   70,    6, 6919,    8, 1163,   70,    6,   43,
         43, 1802, 1802, 1802,   36,   36,   36,   36, 4308, 5410,    4,
        124,  592,  107,   22,    2, 1747, 4065,   27,   10, 1309,   10,
       6414,   10,  190,   10,  416,   10,   27,   10, 1309,   10, 6414,
         10,  190,   10,  416,   10,  456,  139,   15,    7,    2, 4610,
        164,   27,   10, 1309,   10, 6414,   10,  190,   10,  416,   10,
         27,   76,   27, 1309,   76,   27, 6414,   76,   27,  190,   76,
         27], dtype=int32)

# Model Building

In [77]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *

##### Define Model Architecture

In [78]:
# sequential model
model = Sequential()

# embedding layer
model.add(Embedding(x_voc_size, 50, input_length=max_len, mask_zero=True))

# RNN layer
model.add(SimpleRNN(128,activation='relu'))

# dense layer
model.add(Dense(128, activation='relu'))

#output layer
model.add(Dense(10,activation='sigmoid'))

In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           628750    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               22912     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 669,464
Trainable params: 669,464
Non-trainable params: 0
_________________________________________________________________


In [80]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

In [81]:
mc = ModelCheckpoint("weights_best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

## Train Model

In [83]:
history = model.fit(x_train_seq, y_train, batch_size=128, epochs=15, verbose=1, validation_data=(x_valid_seq,y_valid),callbacks=[mc])

Train on 8884 samples, validate on 2222 samples
Epoch 1/15
8884/8884 [==============================] - 10s 1ms/step - loss: 0.5871 - val_loss: 0.4836

Epoch 00001: val_loss improved from inf to 0.48365, saving model to weights_best.hdf5
Epoch 2/15
8884/8884 [==============================] - 9s 980us/step - loss: 0.4773 - val_loss: 0.4738

Epoch 00002: val_loss improved from 0.48365 to 0.47379, saving model to weights_best.hdf5
Epoch 3/15
8884/8884 [==============================] - 9s 965us/step - loss: 0.4485 - val_loss: 0.4378

Epoch 00003: val_loss improved from 0.47379 to 0.43780, saving model to weights_best.hdf5
Epoch 4/15
8884/8884 [==============================] - 9s 980us/step - loss: 0.4105 - val_loss: 0.4276

Epoch 00004: val_loss improved from 0.43780 to 0.42756, saving model to weights_best.hdf5
Epoch 5/15
8884/8884 [==============================] - 8s 951us/step - loss: 0.3927 - val_loss: 0.4302

Epoch 00005: val_loss did not improve from 0.42756
Epoch 6/15
8884/8884 

# Model Predictions

In [85]:
# load weights into new model
model.load_weights("weights_best.hdf5")

#predict probabilities
pred_prob = model.predict(x_valid_seq)

In [86]:
pred_prob[0]

array([0.03611231, 0.01988664, 0.0798845 , 0.26136142, 0.20482484,
       0.00886136, 0.5898737 , 0.53161955, 0.05482775, 0.25114483],
      dtype=float32)

The predictions are in terms of probabilities for each of the 10 tags. Hence we need to have a threshold value to convert these probabilities to 0 or 1.

Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the validation set.

In [92]:
#define candidate threshold values
import numpy as np
threshold  = np.arange(0,0.5,0.01)
threshold

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49])

In [98]:
# Let's define a function that takes a threshold value and uses it to convert probabilities into 1 or 0.
# convert probabilities into classes or tags based on a threshold value

def classify(pred_prob,thresh):
  y_pred_seq = []

  for i in pred_prob:
    temp=[]
    for j in i:
      if j>=thresh:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [100]:
from sklearn import metrics
score=[]

#convert to 1 array
y_true = np.array(y_valid).ravel() 

for thresh in threshold:
    
    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh) 

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [101]:
#find the optimal threshold
opt = threshold[score.index(max(score))]
opt

0.31

#Model Evaluation

In [102]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [103]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87     17520
           1       0.52      0.61      0.56      4700

    accuracy                           0.80     22220
   macro avg       0.71      0.73      0.72     22220
weighted avg       0.81      0.80      0.81     22220



In [104]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_valid))

df = pd.DataFrame({'comment':x_valid,'actual':y_true,'predictions':y_pred})

In [105]:
df.sample(10)

,comment,actual,predictions
1814,is there any way to test whether a series should be logged or transformed in another way i have a code of which i use to run lots of different data through to forecast some of the data definitely ...,"(r, time series)","(classification, machine learning, r)"
1083,i d like to perform a test for homogeneity of variance in r but i don t have samples available to test i have the sample mean of each group the sample deviation of each group and the sample size o...,"(hypothesis testing, r)","(r, regression)"
221,i am currently reading my econometrics notes and there is an example that has really stumped me the example has an answer with it but i do not understand a few things now what i do not understand ...,"(hypothesis testing, self study)","(distributions, probability, self study)"
1495,i have days of daily attendance data and need to use the first days data to predict the next days and match it with my actual values how do i decide which is the best model to use for forecasting ...,"(r, time series)","(r, regression)"
1327,que consider a normal population with unknown mean mu and variance sigma to test h mu against h mu ne a random sample of size is taken based on this sample the test of the form bar x n k rejects t...,"(hypothesis testing, self study)","(distributions, probability, self study)"
2213,for supervised machine learning for prediction if i had some feature variables that are real and also some features that are categorical which have been coded using dummy variables etc i have norm...,"(machine learning, regression)","(logistic, r, regression)"
437,i ve been revising the concepts of logistic regression and suddenly realized that the probability function of the logistic model in the book isl looks absolutely different from other sources for i...,"(logistic, machine learning, probability)","(r, regression)"
1538,i effectively want to model the probability of a player winning his service point a point in which he is the server based on the values of explanatory variables namely court surface and opponent w...,"(logistic, machine learning, r, regression)","(r, regression)"
101,i understand the concept of scaling the data matrix to use in a linear regression model for example in r you could use scaled data scale data scale true my only question is for new observations fo...,"(r, regression)","(r, regression)"
195,say i have a multivariate gaussian mixture model p x sum k pi in mu i sigma i of which the boldsymbol mu and boldsymbol sigma are known so the likelihood function of the coefficients boldsymbol pi...,"(distributions, self study)","(logistic, r, regression)"


# Using LSTM

In [112]:
import keras.backend as k

In [113]:
k.clear_session()

# sequential model
model = Sequential()

# embedding layer
model.add(Embedding(x_voc_size, 50, input_length=max_len, mask_zero=True))

#lstm 
model.add(LSTM(128))

# dense layer
model.add(Dense(128, activation='relu'))

#output layer
model.add(Dense(10,activation='sigmoid'))

In [114]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           628750    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 738,200
Trainable params: 738,200
Non-trainable params: 0
_________________________________________________________________


In [115]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

#checkpoint to save best model during training
mc = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [116]:
history = model.fit(x_train_seq, y_train, batch_size=128, epochs=15, verbose=1, validation_data=(x_valid_seq,y_valid),callbacks=[mc])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8884 samples, validate on 2222 samples
Epoch 1/15
8884/8884 [==============================] - 35s 4ms/step - loss: 0.5127 - val_loss: 0.4829

Epoch 00001: val_loss improved from inf to 0.48295, saving model to weights.best.hdf5
Epoch 2/15
8884/8884 [==============================] - 34s 4ms/step - loss: 0.4815 - val_loss: 0.4776

Epoch 00002: val_loss improved from 0.48295 to 0.47757, saving model to weights.best.hdf5
Epoch 3/15
8884/8884 [==============================] - 33s 4ms/step - loss: 0.4377 - val_loss: 0.4202

Epoch 00003: val_loss improved from 0.47757 to 0.42016, saving model to weights.best.hdf5
Epoch 4/15
8884/8884 [==============================] - 35s 4ms/step - loss: 0.3987 - val_loss: 0.4031

Epoch 00004: val_loss improved from 0.42016 to 0.40306, saving model to weights.best.hdf5
Epoch 5/15
8884/8884 [==============================] - 33s 4ms/step - loss: 0.3570 - val_loss: 0.4288

Epoch 00005: val_loss did not improve from 0.40306
Epoch 6/15
8884/8884 [===

In [118]:
# load weights into new model 
model.load_weights("weights.best.hdf5")

#predict probabilities
pred_prob = model.predict(x_valid_seq)

In [120]:
score=[]

#convert to 1 array
y_true = np.array(y_valid).ravel() 

for thresh in threshold:
    
    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh) 

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [121]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [123]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_valid))

df = pd.DataFrame({'comment':x_valid,'actual':y_true,'predictions':y_pred})
df.sample(10)

,comment,actual,predictions
780,is there any function for m estimation in multivariate linear regression model in r i can estimate the beta s in my model by using the rlm by rewriting the y variables into one column but i would ...,"(r, regression)","(r, regression)"
1905,i have one exam question anyone can answer and explain the reason for me you are studying the following y beta beta x x u you know that the two variables x and x are positively correlated to avoid...,"(regression, self study)","(regression, self study)"
1579,i m exploring some stack overflow data amongst other variables i have variables for the time questions were asked and the time they were answered i m interested in predicting how long a questioner...,"(machine learning, time series)","(r, regression)"
1643,in ecology we often use the logistic growth equation n t frac k n e rt k n e rt or n t frac k n n k n e rt where k is the carrying capacity maximum density reached n is the initial density r is th...,"(distributions, r)","(distributions, hypothesis testing, probability, self study)"
1928,if i have to build a time series prediction model and all i have is a sequence of timestamps of a user when he logs in to a site how do you model that here is the first few rows of the data i have...,"(regression, time series)","(r, time series)"
395,i am constructing a class classifier and using cross validation to tune certain parameters in my model the predictor variables are both continuous and one is ordinal based on looking at the respon...,"(classification, machine learning)","(logistic, r, regression)"
1594,i once heard that log transformation is the most popular one for right skewed distributions in linear regression or quantile regression i would like to know is there any reason underlying this sta...,"(distributions, regression)","(distributions, probability, self study)"
1582,i have an outcome which is truncated from both left and right sides i would like to know which regression method may account this kind of analysis here is an example set seed z rnorm y z z z trunc...,"(r, regression)","(r, regression)"
2119,assume i have a time series x t that i want to fit using an arima model of the form delta x t alpha delta x t w t this could be rewritten as x t x t alpha x t x t w t x t alpha x t alpha x t w t t...,"(r, time series)","(r, time series)"
1344,i am using both r and sas for the time series modeling there is an option in sas that i could not find so far in any packages developed in r for the time series modeling such as tsa or forecast pa...,"(r, time series)","(r, time series)"


# Using GRU

In [124]:
k.clear_session()

# sequential model
model = Sequential()

# embedding layer
model.add(Embedding(x_voc_size, 50, input_length=max_len, mask_zero=True))

#lstm 
model.add(GRU(128))

# dense layer
model.add(Dense(128, activation='relu'))

#output layer
model.add(Dense(10,activation='sigmoid'))

In [125]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

#checkpoint to save best model during training
mc = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#train the model
history = model.fit(x_train_seq, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_valid_seq,y_valid),callbacks=[mc])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8884 samples, validate on 2222 samples
Epoch 1/10
8884/8884 [==============================] - 31s 4ms/step - loss: 0.5271 - val_loss: 0.4800

Epoch 00001: val_loss improved from inf to 0.48004, saving model to weights.best.hdf5
Epoch 2/10
8884/8884 [==============================] - 30s 3ms/step - loss: 0.4744 - val_loss: 0.4717

Epoch 00002: val_loss improved from 0.48004 to 0.47172, saving model to weights.best.hdf5
Epoch 3/10
8884/8884 [==============================] - 30s 3ms/step - loss: 0.4400 - val_loss: 0.4294

Epoch 00003: val_loss improved from 0.47172 to 0.42937, saving model to weights.best.hdf5
Epoch 4/10
8884/8884 [==============================] - 30s 3ms/step - loss: 0.3970 - val_loss: 0.4079

Epoch 00004: val_loss improved from 0.42937 to 0.40792, saving model to weights.best.hdf5
Epoch 5/10
8884/8884 [==============================] - 30s 3ms/step - loss: 0.3729 - val_loss: 0.4013

Epoch 00005: val_loss improved from 0.40792 to 0.40129, saving model to weig

In [127]:
# load weights into new model
model.load_weights("weights.best.hdf5")

#predict probabilities
pred_prob = model.predict(x_valid_seq)

In [128]:
score=[]

#convert to 1 array
y_true = np.array(y_valid).ravel() 

for thresh in threshold:
    
    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh) 

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [129]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [130]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88     17520
           1       0.56      0.69      0.62      4700

    accuracy                           0.82     22220
   macro avg       0.74      0.78      0.75     22220
weighted avg       0.84      0.82      0.83     22220



In [131]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_valid))

df = pd.DataFrame({'comment':x_valid,'actual':y_true,'predictions':y_pred})
df.sample(10)

,comment,actual,predictions
313,when using the step plr function in the stepplr package if my predictors are factors do i need to encode my predictors as dummy variables manually before passing it to the function i do know that ...,"(logistic, r, regression)","(r, regression)"
2182,i m diving into arima models and was trying to repreduce the results of auto regression here is a reproducable example set seed z arima sim n list ar c when running ar without an intercept ceof ar...,"(r, time series)","(r, regression, time series)"
239,my binary logistic regression analysis method enter model has all beta values are statistically non significant should i include these predictors in the final binary logistic regression equation,"(logistic, regression)","(logistic, r, regression)"
1998,my rudimentary workings so far is that log y i x i log y i log x i factorise so log y i x i log y i upsilon i log gamma i thus elasticity of y to x is always however i m not sure if i m correct wh...,"(regression, self study)","(hypothesis testing, machine learning, regression, self study)"
60,i recently had a quiz where i had to find out whether two variables were dependent or independent i know that given f x y if h x g y f x y then they are independent else dependent i am confused ab...,"(probability, self study)","(distributions, probability, self study)"
1882,suppose there is a vector v that contains the body height of every person over years old on earth it looks something like this and is normally distributed also suppose there is simple function f t...,"(distributions, hypothesis testing)","(distributions, probability)"
529,i m looking to create a model that takes into account multiple logistic variables in an ordered process to illustrate what i m trying to do is similar to the following profit option profit option ...,"(logistic, regression)","(regression,)"
561,i was studying on this subject and i got some questions lets take the test h theta in theta space vs space h theta in theta c where theta is the parametric space and lambda x frac sup theta l thet...,"(hypothesis testing, self study)","(distributions, probability, self study)"
1873,in a supervised learning problem i have a training dataset d comprised of samples x and their corresponding labels omega from this data i attempt to learn the true distributions p x omega with som...,"(classification, machine learning)","(classification, machine learning)"
1290,good day i believe this issue is more of a lack of understand of r as i have never used it till recently than anything else what i am looking for is references or documents to help me solve my pro...,"(hypothesis testing, machine learning, r)","(r, regression)"


# USing CNN

In [132]:
# define model architecture
K.clear_session()
model =  Sequential()
model.add(Embedding(x_voc_size, 50, trainable=True, input_shape=(max_len,)))  #embedding layer
  
model.add(Conv1D(64,3,padding='same'))  #conv1d layer
model.add(Dropout(0.1))

model.add(GlobalMaxPooling1D()) 
  
model.add(Dense(128,activation='relu'))  #dense layer

model.add(Dense(10,activation='sigmoid')) #output layer

#summary) of model
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           628750    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 64)           9664      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 648,024
Trainable params: 648,024
Non-trainable params: 0
________________________________________________

In [133]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

#checkpoint to save best model during training
mc = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [134]:
history = model.fit(x_train_seq, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_valid_seq,y_valid),callbacks=[mc])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8884 samples, validate on 2222 samples
Epoch 1/10
8884/8884 [==============================] - 4s 401us/step - loss: 0.5299 - val_loss: 0.4758

Epoch 00001: val_loss improved from inf to 0.47575, saving model to weights.best.hdf5
Epoch 2/10
8884/8884 [==============================] - 3s 353us/step - loss: 0.4293 - val_loss: 0.3650

Epoch 00002: val_loss improved from 0.47575 to 0.36504, saving model to weights.best.hdf5
Epoch 3/10
8884/8884 [==============================] - 3s 382us/step - loss: 0.3264 - val_loss: 0.2997

Epoch 00003: val_loss improved from 0.36504 to 0.29967, saving model to weights.best.hdf5
Epoch 4/10
8884/8884 [==============================] - 3s 375us/step - loss: 0.2772 - val_loss: 0.2774

Epoch 00004: val_loss improved from 0.29967 to 0.27736, saving model to weights.best.hdf5
Epoch 5/10
8884/8884 [==============================] - 4s 396us/step - loss: 0.2499 - val_loss: 0.2715

Epoch 00005: val_loss improved from 0.27736 to 0.27152, saving model to

In [135]:
# load weights into new model
model.load_weights("weights.best.hdf5")

#predict probabilities
pred_prob = model.predict(x_valid_seq)

In [136]:
score=[]

#convert to 1 array
y_true = np.array(y_valid).ravel() 

for thresh in threshold:
    
    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh) 

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [137]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [138]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92     17520
           1       0.69      0.79      0.74      4700

    accuracy                           0.88     22220
   macro avg       0.82      0.85      0.83     22220
weighted avg       0.89      0.88      0.88     22220



In [139]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_valid))

df = pd.DataFrame({'comment':x_valid,'actual':y_true,'predictions':y_pred})
df.sample(10)

,comment,actual,predictions
1850,i am looking for the equation for the se fit values when using logistic regression in r i have seen this answer how are the standard errors computed for the fitted values from a logistic regressio...,"(r, regression)","(logistic, r, regression)"
1808,imagine i have the following data set a b how can i test if a and b have the same trend is the jonckheere s trend test really the test i am looking for i have found this documentation but how woul...,"(hypothesis testing, r)","(hypothesis testing, r, time series)"
223,i am having trouble interpreting the z values for categorical variables in logistic regression in the example below i have a categorical variable with classes and according to the z value class mi...,"(logistic, r)","(logistic, regression)"
626,i an trying to perform a multivariate linear regression in r i have two dependent variables outcomes and one independent variable predictor the model would be something like y y x i did not find a...,"(r, regression)","(r, regression)"
1343,i am writing a program to predict the click through rates of online ads two important notes about this problem click through rates are very small like click through rates depend on several paramet...,"(logistic, machine learning)","(classification, machine learning)"
546,i conducted an experiment with three treatments a b control measuring for each subject a response variable that varies between and a continuous proportion the resulting distributions of the respon...,"(distributions, hypothesis testing)","(distributions, hypothesis testing, r)"
735,i ve done a cross correlation but i m not sure if what i m doing is correct here are the command lines i ve used so far this one is for the first time series it represents the numbers of subscript...,"(r, time series)","(r, time series)"
833,how to prove that distribution function of kolmogorov distribution is a distribution function in particular that it is in infinity and is increasing function f x begin cases sum limits k infty inf...,"(distributions, probability)","(distributions, probability)"
32,i would like to monitor customer satisfaction over time and would ideally like to use nps for that specifically i would like to see if there s an overall trend over time could i regress net promot...,"(regression, time series)","(hypothesis testing, r)"
1371,i am a non student working through the first edition of yates and goodman s text probability and stochastic processes on page question goes like this suppose you arrive at a bus stop at time and t...,"(distributions, self study)","(distributions, probability)"
